##### AJUSTA T1 PRA TAMANHO DO FLAIR

In [ ]:
import nibabel as nib
import numpy as np
from scipy.ndimage import zoom

flair_path = 'Displasia/sub-00003/anat/sub-00003_acq-T2sel_FLAIR.nii.gz'
t1_path = 'Displasia/sub-00003/anat/sub-00003_acq-iso08_T1w.nii.gz' 
output_path = 't1_reduzido.nii.gz' # Caminho onde quer salvar

# Paths das imagens
flair_img = nib.load(flair_path)
t1_img = nib.load(t1_path)

# Data das imagens
flair_data = flair_img.get_fdata()
t1_data = t1_img.get_fdata()

# Shape das imagens
flair_shape = flair_data.shape
t1_shape = t1_data.shape

# Calcular o fator de escala para redimensionar a imagem T1 para o tamanho da imagem FLAIR
scaling_factor = np.array(flair_shape) / np.array(t1_shape)

# Redimensionar a imagem T1 para as dimensões da imagem FLAIR usando interpolação trilinear
t1_rescaled = zoom(t1_data, scaling_factor, order=1)  # order=1 para interpolação bilinear

# Salvar a imagem T1 escalada
t1_rescaled_img = nib.Nifti1Image(t1_rescaled, t1_img.affine)
nib.save(t1_rescaled_img, output_path) # Implementar lógica para salvar a imagem redimensionada e já usá-la na próxima célula

##### VISUALIZAÇÃO DE IMAGENS

In [ ]:
import matplotlib.pyplot as plt
from nilearn.image import load_img
import os

img_path = "C:/Users/Team Taiane/Desktop/ADNI/FULL_ADNI/NIFTI_PROCESSED/test/cn/I495042.nii.gz"

# Definir as coordenadas de corte
x_slices = [60, 90, 120]  # Coordenadas para as fatias no eixo x
y_slices = [60, 90, 120]  # Coordenadas para as fatias no eixo y
z_slices = [60, 90, 120]  # Coordenadas para as fatias no eixo z

# Carregar a imagem do cérebro e a máscara de lesão
img_brain = load_img(img_path)

# Obter os dados das imagens como arrays numpy
brain_data = img_brain.get_fdata()

print(f"{os.path.basename(img_path)} - shape: {brain_data.shape}")

# Criar a figura para plotagem
fig, axes = plt.subplots(3, 3, figsize=(20, 10))  # Plotando 3 linhas e 6 colunas para múltiplas fatias

# Plotar as fatias para o eixo x
for i, x in enumerate(x_slices):
    brain_sagittal = brain_data[x, :, :]  # Fatia sagital no eixo x
    axes[0, i].imshow(brain_sagittal.T, cmap="gray", origin="lower")  # Exibe a fatia do cérebro
    axes[0, i].set_title(f"Fatia Sagital x={x}")
    axes[0, i].axis("off")  # Desativar os eixos

# Plotar as fatias para o eixo y
for i, y in enumerate(y_slices):
    brain_coronal = brain_data[:, y, :]  # Fatia coronal no eixo y
    axes[1, i].imshow(brain_coronal.T, cmap="gray", origin="lower")  # Exibe a fatia do cérebro
    axes[1, i].set_title(f"Fatia Coronal y={y}")
    axes[1, i].axis("off")  # Desativar os eixos

# Plotar as fatias para o eixo z
for i, z in enumerate(z_slices):
    brain_axial = brain_data[:, :, z]  # Fatia axial no eixo z
    axes[2, i].imshow(brain_axial.T, cmap="gray", origin="lower")  # Exibe a fatia do cérebro
    axes[2, i].set_title(f"Fatia Axial z={z}")
    axes[2, i].axis("off")  # Desativar os eixos

# Ajustar o layout para garantir que os títulos não se sobreponham
plt.tight_layout()
plt.show()

##### DOWNSAMPLING (DICOM)

In [ ]:
import os
import shutil
import random
from math import ceil, floor

def explore_folder(directory, string): # função para percorrer cada diretório e retornar todos os caminhos de .nii.gz
    paths =  []
    for item in os.listdir(directory):
        folder = os.path.join(directory, item, 'ADNI')
        for subject in os.listdir(folder):
            patient = os.path.join(folder, subject, string)
            for date in os.listdir(patient):
                exam = os.path.join(patient, date)
                for file in os.listdir(exam):
                    file_path = os.path.join(exam, file)
                    paths.append(file_path)
    return paths

base = 'MPRAGE'
classification = 'ad' 
directory = os.path.join(base, classification)

# pra guardar os endereços dos arquivos da classe
paths = []

# testar se há subpastas vazias
empty_subjects = 0 
empty_exams = 0

# enche o vetor
paths = explore_folder(directory, 'MPRAGE')

#print(paths)

print(f"Total de arquivos: {len(paths)}")
print(f"Total de exames vazias: {empty_exams}")
print(f"Total de pacientes vazias: {empty_subjects}")

total_paths = len(paths)

target_count = 735 # valor para o qual deseja-se reduzir o número de amostras

random.shuffle(paths) # embaralha vetor para fazer remoção sem viés

# remover até que a lista tenha target_count itens
while total_paths > target_count:
    path_to_remove = paths.pop()

    #shutil.rmtree(path_to_remove)  # Remove o diretório (com seus conteúdos)
    print(f"Pasta removida: {path_to_remove}")
    
    total_paths -= 1  # decrementa a quantidade de arquivos/pastas removidos

# agora, a lista de paths deve ter target_count itens restantes
print(f"Restando {len(paths)} arquivos/pastas.")

##### DIVISÃO ENTRE CONJUNTOS (DICOM)

In [ ]:
group = 'ad' # classe a ser dividida
cont = 0

# caminhos de cada conjunto
train_folder = os.path.join('conjuntos', 'train', group)
val_folder = os.path.join('conjuntos', 'validation', group)
test_folder = os.path.join('conjuntos', 'test', group)

# diretórios usados com os arquivos espalhados
mp_dir = os.path.join('MPRAGE', group)
sense_dir = os.path.join('SENSE2', group)

# vetores para armazenar os caminhos já divididos entre os conjuntos
train_paths = []
val_paths = []
test_paths = []

# retornar os caminhos para cada arquivo .nii.gz espalhado
mp_paths = explore_folder(mp_dir, 'MPRAGE')
sense_paths = explore_folder(sense_dir, 'MPRAGE_SENSE2')
full_paths = mp_paths + sense_paths

# calcular quantos arquivos vão pra cada conjunto
n = len(full_paths)
train = floor(n*0.7)
validation = floor(n*0.2)
test = floor(n*0.1)

print(f"full: {n}\ntrain (70): {train}\nvalidation (20): {validation}\ntest (10): {test}\nsoma: {train+validation+test}\n\n")

# enche cada vetor
for path in full_paths:
    if cont < train:
        cont += 1
        train_paths.append(path)
    elif cont < train+validation:
        cont += 1
        val_paths.append(path)
    else:
        test_paths.append(path)

print(f"train (70): {len(train_paths)}\nvalidation (20): {len(val_paths)}\ntest (10): {len(test_paths)}\nsoma: {len(train_paths)+len(val_paths)+len(test_paths)}\n\n")

# copia o item do endereço antigo para um novo
for item in train_paths:
    shutil.copytree(item, os.path.join(train_folder, os.path.basename(item)))
for item in val_paths:
    shutil.copytree(item, os.path.join(val_folder, os.path.basename(item)))
for item in test_paths:
    shutil.copytree(item, os.path.join(test_folder, os.path.basename(item)))

##### CROPPAR IMAGENS (NIFTI)

In [1]:
import os
import nibabel as nib
import matplotlib.pyplot as plt

# índices de corte para o corte NIfTI
SLICE_NII_IDX0 = slice(24, 169)
SLICE_NII_IDX1 = slice(24, 206)
SLICE_NII_IDX2 = slice(6, 161)

base = 'train'
dir = 'ad'

for subset in os.listdir(base):
    dir = os.path.join(base, subset) # salva junção do conjunto com a pasta referente à classe
    for name in os.listdir(dir):
        file = os.path.join(dir, name) # acessa cada arquivo dentro da pasta
        image = nib.load(file)
        data = image.get_fdata()

        print(f"shape da imagem {name} original: {data.shape}")
        data_cropped = data[SLICE_NII_IDX0, SLICE_NII_IDX1, SLICE_NII_IDX2] # corta a imagem para o tamanho desejadao
        print(f"shape da imagem {name} cortada: {data_cropped.shape}\n\n")

        # criar um novo objeto NIfTI com o corte
        nifti_cropped = nib.Nifti1Image(data_cropped, image.affine, image.header)
        
        # salvar no mesmo arquivo (sobrescrevendo)
        nib.save(nifti_cropped, file)